<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#7Z1018,-Week-5:-Data-Integration" data-toc-modified-id="7Z1018,-Week-5:-Data-Integration-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>7Z1018, Week 5: Data Integration</a></span><ul class="toc-item"><li><span><a href="#Concatenating-(Stacking)-DataFrames-with-Index-Alignment" data-toc-modified-id="Concatenating-(Stacking)-DataFrames-with-Index-Alignment-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Concatenating (Stacking) DataFrames with Index Alignment</a></span></li><li><span><a href="#The-Employee-Example-DB" data-toc-modified-id="The-Employee-Example-DB-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>The Employee Example DB</a></span></li><li><span><a href="#Joining-DataFrames" data-toc-modified-id="Joining-DataFrames-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Joining DataFrames</a></span></li><li><span><a href="#Checkpoint" data-toc-modified-id="Checkpoint-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Checkpoint</a></span></li></ul></li></ul></div>

# 7Z1018, Week 5: Data Integration

* Author: Luciano Gerber
* Unit: 7Z1018, Introduction to Data Science
* Date: Week 5, 23-Oct-19

In a wide range of situations, a data scientist will find data of interest is found in separate, but related datasets. Usually, for more effective data analysis and modelling (particularly in machine learning), one must merge these into a single, combined dataset. 

Take the example of analysis of climate variation and change in a certain geographical region of the planet. The necessary data weather measurements (e.g., daily temperatures) might have been collected separately by weather stations placed across the region. In order to gain broader insights about the data, those individual datasets must be combined.

Another scenario assumes that there are separate datasets for students and units, and one needs to query information that comes from both (e.g., unit's number of credits and student name). Typically, keys (e.g., `student_id`) are relied upon in the integration procedure.

It has beem claimed said that about 80% of a data scientist's time and effort goes into data wrangling, and integration is part of that. All sorts of challenges one might face here, such as datasets with unmatching and inconsistent schemas, data types, ranges, and units of measurement, to mention a few. 

We firstly review `.concat` and index alignment for combining datasets, and then explore `.merge` for joining datasets that are related by _keys_. Another example database, the `employee_db`, is introduced.

The usual preamble for importing the essential modules and configuring the plotting engine.

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
sns.set(style="ticks", color_codes=True)
sns.set_context("notebook")

In [2]:
sns.set({ "figure.figsize": (12/1.5,8/1.5) })

We will make use of `gapminder` for illustration. As usual, note that the path to the data file should be changed to match the directory structure in your home area.

In [3]:
gm_df = pd.read_csv('gapminder.tsv', sep='\t')

A quick check:

In [4]:
gm_df.sample()

,country,continent,year,lifeExp,pop,gdpPercap
790,Jamaica,Americas,2002,72.047,2664659,6994.774861


## Concatenating (Stacking) DataFrames with Index Alignment

Let us take a slice of the gapminder dataset for life expectancy in the `Americas` in `1952`.

In [5]:
gm_1952_lifeExp_df = gm_df.loc[ 
    (gm_df['year']==1952) & (gm_df['continent']=='Americas'),
    [ 'country', 'lifeExp' ] 
]
gm_1952_lifeExp_df = gm_1952_lifeExp_df\
    .set_index('country')\
    .rename(columns={'lifeExp': 'lifeExp_1952'})

gm_1952_lifeExp_df.head()

,lifeExp_1952
country,
Argentina,62.485
Bolivia,40.414
Brazil,50.917
Canada,68.750
Chile,54.745


And a similar slice from `2007`, which we can be used for comparison.

In [6]:
gm_2007_lifeExp_df = gm_df.loc[ 
    (gm_df['year']==2007) & (gm_df['continent']=='Americas'), 
    [ 'country', 'lifeExp' ]
]
gm_2007_lifeExp_df = gm_2007_lifeExp_df\
    .set_index('country')\
    .rename(columns={'lifeExp': 'lifeExp_2007'})

gm_2007_lifeExp_df.head()

,lifeExp_2007
country,
Argentina,75.320
Bolivia,65.554
Brazil,72.390
Canada,80.653
Chile,78.553


As we've seen before, one can use `pd.concat()` for concatenating a list of DataFrames (or Series) according to matching index labels on the axis it operates. Here, we are stacking DataFrames horizontally, i.e., across columns. 

In [7]:
pd.concat([gm_1952_lifeExp_df, gm_2007_lifeExp_df], axis=1).head()

,lifeExp_1952,lifeExp_2007
country,,
Argentina,62.485,75.320
Bolivia,40.414,65.554
Brazil,50.917,72.390
Canada,68.750,80.653
Chile,54.745,78.553


This functionality is quite useful for some quick comparisons of individual groups of data. For example, we can take the statistical summary of life expectancy for both `1952` and `2007` for contrast.

In [8]:
summ_2007_lifeExp_df = gm_df.loc[gm_df['year']==2007, 'lifeExp'].describe()
summ_2007_lifeExp_df.name = 'summ_2007_lifeExp_df'
summ_2007_lifeExp_df

count    142.000000
mean      67.007423
std       12.073021
min       39.613000
25%       57.160250
50%       71.935500
75%       76.413250
max       82.603000
Name: summ_2007_lifeExp_df, dtype: float64

In [9]:
summ_1952_lifeExp_df = gm_df.loc[gm_df['year']==1952, 'lifeExp'].describe()
summ_1952_lifeExp_df.name = 'summ_1952_lifeExp_df'
summ_1952_lifeExp_df

count    142.000000
mean      49.057620
std       12.225956
min       28.801000
25%       39.059000
50%       45.135500
75%       59.765000
max       72.670000
Name: summ_1952_lifeExp_df, dtype: float64

It is important to specify the proper concatenation axis - see what happens with the default option.

In [10]:
pd.concat([summ_1952_lifeExp_df, summ_2007_lifeExp_df])

count    142.000000
mean      49.057620
std       12.225956
min       28.801000
25%       39.059000
50%       45.135500
75%       59.765000
max       72.670000
count    142.000000
mean      67.007423
std       12.073021
min       39.613000
25%       57.160250
50%       71.935500
75%       76.413250
max       82.603000
dtype: float64

In [11]:
df = pd.concat([summ_1952_lifeExp_df, summ_2007_lifeExp_df], axis=1)
df

,summ_1952_lifeExp_df,summ_2007_lifeExp_df
count,142.000000,142.000000
mean,49.057620,67.007423
std,12.225956,12.073021
min,28.801000,39.613000
25%,39.059000,57.160250
50%,45.135500,71.935500
75%,59.765000,76.413250
max,72.670000,82.603000


In [12]:
perc_increase = (df['summ_2007_lifeExp_df']-df['summ_1952_lifeExp_df'])/df['summ_1952_lifeExp_df']*100
perc_increase.name = 'perc_increase_1952_2007'
perc_increase

count     0.000000
mean     36.589225
std      -1.250906
min      37.540363
25%      46.343352
50%      59.376766
75%      27.856187
max      13.668639
Name: perc_increase_1952_2007, dtype: float64

What happens when the row indices do not overlap fully? We've seen it before - `NaN`s.

In [13]:
df1 = gm_1952_lifeExp_df.loc['Argentina':'Canada']
df1

,lifeExp_1952
country,
Argentina,62.485
Bolivia,40.414
Brazil,50.917
Canada,68.750


In [14]:
df2 = gm_2007_lifeExp_df.loc['Brazil':'Chile']
df2

,lifeExp_2007
country,
Brazil,72.390
Canada,80.653
Chile,78.553


In [15]:
pd.concat([df1, df2], axis=1, sort=True)

,lifeExp_1952,lifeExp_2007
Argentina,62.485,NaN
Bolivia,40.414,NaN
Brazil,50.917,72.390
Canada,68.750,80.653
Chile,NaN,78.553


## The Employee Example DB

The zip file containing the set of `.csv` files can be obtained from. As usual, decompress it and place the relevant datasets into a suitable directory in your home.

Briefly, we have `employees` that work for `departments` based in certain `locations` that exist in `countries` which are placed in `regions`. 

In [16]:
employees_df = pd.read_csv('employee_db/employees.csv')
employees_df.head()

,employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,commission_pct,manager_id,department_id
0,100,Steven,King,SKING,515.123.4567,1987-06-17,AD_PRES,24000.0,NaN,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,1989-09-21,AD_VP,17000.0,NaN,100.0,90.0
2,102,Lex,De Haan,LDEHAAN,515.123.4569,1993-01-13,AD_VP,17000.0,NaN,100.0,90.0
3,103,Alexander,Hunold,AHUNOLD,590.423.4567,1990-01-03,IT_PROG,9000.0,NaN,102.0,60.0
4,104,Bruce,Ernst,BERNST,590.423.4568,1991-05-21,IT_PROG,6000.0,NaN,103.0,60.0


In [17]:
departments_df = pd.read_csv('employee_db/departments.csv')
departments_df.head()

,department_id,department_name,manager_id,location_id
0,10,Administration,200.0,1700
1,20,Marketing,201.0,1800
2,30,Purchasing,114.0,1700
3,40,Human Resources,203.0,2400
4,50,Shipping,121.0,1500


In [18]:
locations_df = pd.read_csv('employee_db/locations.csv')
locations_df.head()

,location_id,street_address,postal_code,city,state_province,country_id
0,1000,1297 Via Cola di Rie,00989,Roma,NaN,IT
1,1100,93091 Calle della Testa,10934,Venice,NaN,IT
2,1200,2017 Shinjuku-ku,1689,Tokyo,Tokyo Prefecture,JP
3,1300,9450 Kamiya-cho,6823,Hiroshima,NaN,JP
4,1400,2014 Jabberwocky Rd,26192,Southlake,Texas,US


In [19]:
countries_df = pd.read_csv('employee_db/countries.csv')
countries_df.head()

,country_id,country_name,region_id
0,IT,Italy,1
1,JP,Japan,3
2,US,United States of America,2
3,CA,Canada,2
4,CN,China,3


In [20]:
regions_df = pd.read_csv('employee_db/regions.csv')
regions_df.head()

,region_id,region_name
0,1,Europe
1,2,Americas
2,3,Asia
3,4,Middle East and Africa


## Joining DataFrames

In many situations, following good practices of _relational database design_ to, for example, prevent issues with data replication and update anomalies, data is spread across different datasets that are related by __keys__. In relational database's terminology, one dataset's __foreign key__ links to a dataset's __primary key__ (the latter, a unique identifier for each) instance of data.

For example, `employees(department_id)` is a foreign key to `departments(department_id)`. Those columns and their matching values are used to keep records (e.g., rows) from both datasets related.

In such cases, more often than not, _interesting questions_ about the data require __joining datasets__. With `pandas`, a natural way of achieving this is via the method `.merge`.

In [21]:
countries_df.head()

,country_id,country_name,region_id
0,IT,Italy,1
1,JP,Japan,3
2,US,United States of America,2
3,CA,Canada,2
4,CN,China,3


In [22]:
regions_df.head()

,region_id,region_name
0,1,Europe
1,2,Americas
2,3,Asia
3,4,Middle East and Africa


In [23]:
countries_df.merge(regions_df).sample(5)

,country_id,country_name,region_id,region_name
16,BR,Brazil,2,Americas
22,KW,Kuwait,4,Middle East and Africa
10,IN,India,3,Asia
3,DE,Germany,1,Europe
8,JP,Japan,3,Asia


It is always good practice to check the columns of each DataFrame to verify which columns are the keys, and how they are named.

In [24]:
employees_df.columns

Index(['employee_id', 'first_name', 'last_name', 'email', 'phone_number',
       'hire_date', 'job_id', 'salary', 'commission_pct', 'manager_id',
       'department_id'],
      dtype='object')

In [25]:
departments_df.columns

Index(['department_id', 'department_name', 'manager_id', 'location_id'], dtype='object')

In [26]:
departments_df.columns & employees_df.columns

Index(['department_id', 'manager_id'], dtype='object')

The following is an example of joining two DataFrames on a specified column they have in common rows from the two DataFrames are concatenated based on matching values of their joining column(s) (key(s)):

In [27]:
employees_df.merge(departments_df, on='department_id').head()

,employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,commission_pct,manager_id_x,department_id,department_name,manager_id_y,location_id
0,100,Steven,King,SKING,515.123.4567,1987-06-17,AD_PRES,24000.0,NaN,NaN,90.0,Executive,100.0,1700
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,1989-09-21,AD_VP,17000.0,NaN,100.0,90.0,Executive,100.0,1700
2,102,Lex,De Haan,LDEHAAN,515.123.4569,1993-01-13,AD_VP,17000.0,NaN,100.0,90.0,Executive,100.0,1700
3,103,Alexander,Hunold,AHUNOLD,590.423.4567,1990-01-03,IT_PROG,9000.0,NaN,102.0,60.0,IT,103.0,1400
4,104,Bruce,Ernst,BERNST,590.423.4568,1991-05-21,IT_PROG,6000.0,NaN,103.0,60.0,IT,103.0,1400


For reducing clutter and help the illustration, I'll drop some employee attributes for the time being.

In [28]:
employees_sub_df = employees_df[
    ['employee_id', 'first_name', 'last_name', 'salary', 'department_id']
]
employees_sub_df.head()

,employee_id,first_name,last_name,salary,department_id
0,100,Steven,King,24000.0,90.0
1,101,Neena,Kochhar,17000.0,90.0
2,102,Lex,De Haan,17000.0,90.0
3,103,Alexander,Hunold,9000.0,60.0
4,104,Bruce,Ernst,6000.0,60.0


In [29]:
employees_sub_df.merge(departments_df, on='department_id').head()

,employee_id,first_name,last_name,salary,department_id,department_name,manager_id,location_id
0,100,Steven,King,24000.0,90.0,Executive,100.0,1700
1,101,Neena,Kochhar,17000.0,90.0,Executive,100.0,1700
2,102,Lex,De Haan,17000.0,90.0,Executive,100.0,1700
3,103,Alexander,Hunold,9000.0,60.0,IT,103.0,1400
4,104,Bruce,Ernst,6000.0,60.0,IT,103.0,1400


What if names of columns do not match?

In [30]:
departments_id_df = departments_df.rename(columns={'department_id': 'id'})
departments_id_df.head()

,id,department_name,manager_id,location_id
0,10,Administration,200.0,1700
1,20,Marketing,201.0,1800
2,30,Purchasing,114.0,1700
3,40,Human Resources,203.0,2400
4,50,Shipping,121.0,1500


...then, use the options `left_on=` and `right_on=`.

In [31]:
employees_sub_df.merge(departments_id_df, left_on='department_id', right_on='id').head()

,employee_id,first_name,last_name,salary,department_id,id,department_name,manager_id,location_id
0,100,Steven,King,24000.0,90.0,90,Executive,100.0,1700
1,101,Neena,Kochhar,17000.0,90.0,90,Executive,100.0,1700
2,102,Lex,De Haan,17000.0,90.0,90,Executive,100.0,1700
3,103,Alexander,Hunold,9000.0,60.0,60,IT,103.0,1400
4,104,Bruce,Ernst,6000.0,60.0,60,IT,103.0,1400


And if one of the matching parts is actually the row index?

In [32]:
departments_i_df = departments_df.set_index('department_id')
departments_i_df.head()

,department_name,manager_id,location_id
department_id,,,
10,Administration,200.0,1700
20,Marketing,201.0,1800
30,Purchasing,114.0,1700
40,Human Resources,203.0,2400
50,Shipping,121.0,1500


In [33]:
employees_sub_df.merge(departments_i_df, left_on='department_id', right_index=True).head()

,employee_id,first_name,last_name,salary,department_id,department_name,manager_id,location_id
0,100,Steven,King,24000.0,90.0,Executive,100.0,1700
1,101,Neena,Kochhar,17000.0,90.0,Executive,100.0,1700
2,102,Lex,De Haan,17000.0,90.0,Executive,100.0,1700
3,103,Alexander,Hunold,9000.0,60.0,IT,103.0,1400
4,104,Bruce,Ernst,6000.0,60.0,IT,103.0,1400


It is worth pointing out that `.merge` takes **two DataFrames as input** and produces a **new DataFrame as output**. As with other operations on DataFrames, one could **chain** `.merge` calls for joining multiple datasets.

## Checkpoint

Produce a DataFrame that shows the departments together with the addresses of their locations.

In [34]:
dep_loc = departments_i_df.merge(locations_df)
dep_loc.set_index('department_name')[['location_id','street_address','postal_code','city','state_province','country_id']]

,location_id,street_address,postal_code,city,state_province,country_id
department_name,,,,,,
Administration,1700,2004 Charade Rd,98199,Seattle,Washington,US
Purchasing,1700,2004 Charade Rd,98199,Seattle,Washington,US
Executive,1700,2004 Charade Rd,98199,Seattle,Washington,US
Finance,1700,2004 Charade Rd,98199,Seattle,Washington,US
Accounting,1700,2004 Charade Rd,98199,Seattle,Washington,US
Treasury,1700,2004 Charade Rd,98199,Seattle,Washington,US
Corporate Tax,1700,2004 Charade Rd,98199,Seattle,Washington,US
Control And Credit,1700,2004 Charade Rd,98199,Seattle,Washington,US
Shareholder Services,1700,2004 Charade Rd,98199,Seattle,Washington,US


Produce a DataFrame containing the name of the department and the cities in which they are based.

In [35]:
dep_loc.set_index('department_name')[['city']]

,city
department_name,
Administration,Seattle
Purchasing,Seattle
Executive,Seattle
Finance,Seattle
Accounting,Seattle
Treasury,Seattle
Corporate Tax,Seattle
Control And Credit,Seattle
Shareholder Services,Seattle


Produce a DataFrame with the name of the department and the name of the country in which they are based.

In [36]:
dep_loc.set_index('department_name')[['country_id']]

,country_id
department_name,
Administration,US
Purchasing,US
Executive,US
Finance,US
Accounting,US
Treasury,US
Corporate Tax,US
Control And Credit,US
Shareholder Services,US


Produce a DataFrame with the name of the department and the name of the region of the world in which they are based.


In [37]:
dep_loc.set_index('department_name')[['state_province']]

,state_province
department_name,
Administration,Washington
Purchasing,Washington
Executive,Washington
Finance,Washington
Accounting,Washington
Treasury,Washington
Corporate Tax,Washington
Control And Credit,Washington
Shareholder Services,Washington


Produce a DataFrame with the top-5 earners in the IT and Sales department.

In [38]:
result = employees_df.merge(departments_df).set_index('employee_id')[['first_name','last_name','salary','department_name']]
result.loc [
    (result['department_name']=='IT') |
    (result['department_name']=='Sales') 
].sort_values(by='salary', ascending=False).head()

,first_name,last_name,salary,department_name
employee_id,,,,
150,Peter,Tucker,10000.0,Sales
151,David,Bernstein,9500.0,Sales
152,Peter,Hall,9000.0,Sales
153,Christopher,Olsen,8000.0,Sales
154,Nanette,Cambrault,7500.0,Sales


In [39]:
#another way of doing it
employees_df\
    .merge(departments_df, on='department_id')\
    .query('department_name=="IT" or department_name=="Sales"')\
    .sort_values('salary')\
    .tail()

,employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,commission_pct,manager_id_x,department_id,department_name,manager_id_y,location_id
68,148,Gerald,Cambrault,GCAMBRAU,011.44.1344.619268,1999-10-15,SA_MAN,11000.0,0.30,100.0,80.0,Sales,145.0,2500
88,168,Lisa,Ozer,LOZER,011.44.1343.929268,1997-03-11,SA_REP,11500.0,0.25,148.0,80.0,Sales,145.0,2500
67,147,Alberto,Errazuriz,AERRAZUR,011.44.1344.429278,1997-03-10,SA_MAN,12000.0,0.30,100.0,80.0,Sales,145.0,2500
66,146,Karen,Partners,KPARTNER,011.44.1344.467268,1997-01-05,SA_MAN,13500.0,0.30,100.0,80.0,Sales,145.0,2500
65,145,John,Russell,JRUSSEL,011.44.1344.429268,1996-10-01,SA_MAN,14000.0,0.40,100.0,80.0,Sales,145.0,2500


Produce a DataFrame with the top-10 earners in United Kingdom. In the output, include only the employee's name and salary, and the name of the deparment they work for.

In [40]:
result2 = employees_df.merge(departments_df, on='department_id').set_index('employee_id')
result2 = result2.merge(locations_df,on='location_id')
result2 = result2[['first_name','last_name','salary','department_name','country_id']]
result2.loc [
    result2['country_id']=='UK'
].drop('country_id',axis=1).sort_values(by='salary', ascending=False).head(10)

,first_name,last_name,salary,department_name
68,John,Russell,14000.0,Sales
69,Karen,Partners,13500.0,Sales
70,Alberto,Errazuriz,12000.0,Sales
91,Lisa,Ozer,11500.0,Sales
71,Gerald,Cambrault,11000.0,Sales
97,Ellen,Abel,11000.0,Sales
85,Clara,Vishney,10500.0,Sales
72,Eleni,Zlotkey,10500.0,Sales
92,Harrison,Bloom,10000.0,Sales
79,Janette,King,10000.0,Sales


In [41]:
#another way of doing it
employees_df\
    .merge(departments_df, on='department_id')\
    .merge(locations_df, on='location_id')\
    .merge(countries_df, on='country_id')\
    .query('country_name=="United Kingdom"')\
    .sort_values('salary')\
    .tail(10)\
    [['first_name','last_name','salary','department_name']]

,first_name,last_name,salary,department_name
73,Peter,Tucker,10000.0,Sales
92,Harrison,Bloom,10000.0,Sales
85,Clara,Vishney,10500.0,Sales
72,Eleni,Zlotkey,10500.0,Sales
97,Ellen,Abel,11000.0,Sales
71,Gerald,Cambrault,11000.0,Sales
91,Lisa,Ozer,11500.0,Sales
70,Alberto,Errazuriz,12000.0,Sales
69,Karen,Partners,13500.0,Sales
68,John,Russell,14000.0,Sales


How many employess work in Canada?

In [42]:
len(result2.loc[result2['country_id']=='CA'])

2